# Clamped Reissner-Mindlin plate under uniform load using TDNNS finite element method

This demo program solves the out-of-plane Reissner-Mindlin equations on the
unit square with uniform transverse loading with fully clamped boundary
conditions using the Tangential-Displacement-Normal-Normal-Stress (TDNNS)
method described in Pechstein and Schöberl [ref].

It is assumed the reader understands most of the basic functionality
of the new FEniCSx Project.

This demo illustrates how to:

We begin by importing the necessary functionality from DOLFINx, UFL and PETSc.

In [2]:
import numpy as np

from mpi4py import MPI
from petsc4py import PETSc

import dolfinx
from dolfinx import UnitSquareMesh, Function, FunctionSpace, DirichletBC, Constant
from dolfinx.cpp.mesh import CellType
from dolfinx.fem import (apply_lifting, assemble_matrix, assemble_vector,
                         locate_dofs_topological, set_bc)
from dolfinx.mesh import locate_entities_boundary
from dolfinx.io import XDMFFile

import ufl
from ufl import FiniteElement, MixedElement, VectorElement
from ufl import sym, grad, tr, dx, inner, split

We then create a two-dimensional mesh of the mid-plane of the plate $\Omega = [0, 1] \times [0, 1]$. `GhostMode.shared_facet` is required.

In [3]:
mesh = UnitSquareMesh(MPI.COMM_WORLD, 32, 32, CellType.triangle,
                      dolfinx.cpp.mesh.GhostMode.shared_facet)

The first-order TDNNS element for the Reissner-Mindlin plate problem consists of:

- the first-order tensor-valued $H(\mathrm{div} \; \mathrm{div})$-conforming rotated Regge element for the bending moments $M \in \mathrm{SREG}_1$,
- the first-order vector-valued Nedéléc element of the second-kind for the rotation field $\theta \in
  \mathrm{NED}_1^2$ and,
- a second-order scalar-valued Lagrange element for the transverse
  displacement field $w \in \mathrm{CG}_2$.

The final element definition is

In [7]:
U_el = MixedElement([FiniteElement("Regge", ufl.triangle, 1), FiniteElement("N2curl", ufl.triangle, 1),
                     FiniteElement("Lagrange", ufl.triangle, 2)])
U = FunctionSpace(mesh, U_el)

u_ = Function(U)
u = ufl.TrialFunction(U)
u_t = ufl.TestFunction(U)

m_, theta_, w_ = split(u_)

We assume constant material parameters; Young's modulus $E$, Poisson's ratio $\nu$, shear-correction factor $\kappa$, and thickness
$t$.

In [9]:
E = 10920.0
nu = 0.3
kappa = 5.0/6.0
t = 0.001

In [ ]:
# TODO: Rewrite bending energy in terms of compliance tensor and bending moment
D = (E*t**3)/(12.0*(1.0 - nu**2))
psi_b = 0.5*D*((1.0 - nu)*tr(k*k) + nu*(tr(k))**2)

In [6]:
gamma = grad(w_) - theta_
psi_s = ((E*kappa*t)/(4.0*(1.0 + nu)))*inner(gamma_, gamma_)

In [7]:
W_ext = inner(1.0*t**3, w_)*dx

The shear strain vector $\gamma$ can be expressed in terms
of the rotation and transverse displacement field

$$\gamma(\theta, w) = \nabla w - \theta$$

or in UFL

In [8]:
gamma = grad(w_) - theta_

We can now define our Lagrangian for the complete system and derive the residual and Jacobian automatically using the standard UFL `derivative` function

In [10]:
Pi = 
F = ufl.derivative(Pi, u_, u_t)
J = ufl.derivative(F, u_, u)

In the following we use standard from `dolfinx` to apply boundary conditions, assemble, solve and output the solution.

In [11]:
# TODO: Boundary conditions

A = dolfinx.fem.assemble_matrix(J, bcs=bcs)
A.assemble()

b = dolfinx.fem.assemble_vector(-F)
dolfinx.fem.apply_lifting(b, [J], [bcs])
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
dolfinx.fem.set_bc(b, bcs)

ksp = PETSc.KSP().create(MPI.COMM_WORLD)

pc = ksp.getPC()
pc.setType("lu")
pc.setFactorSolverType("mumps")

ksp.setOperators(A)
ksp.setType("preonly")
ksp.setFromOptions()
ksp.solve(b, u_.vector)

bb_tree = dolfinx.cpp.geometry.BoundingBoxTree(mesh, 2)
point = np.array([0.5, 0.5, 0.0], dtype=np.float64)
cell_candidates = dolfinx.cpp.geometry.compute_collisions_point(bb_tree, point)
cell = dolfinx.cpp.geometry.select_colliding_cells(
    mesh, cell_candidates, point, 1)

theta, w, R_gamma, p = u_.split()

if len(cell) > 0:
    value = w.eval(point, cell)
    print(value[0])
    # NOTE: FEniCS-Shells (old dolfin) `demo/documented/reissner-mindlin-clamped` 
    # gives 1.28506469462e-06 on a 32 x 32 mesh and 1.2703580973e-06 on a 64 x 64
    # mesh.
    def test_center_displacement():
        assert(np.isclose(value[0], 1.285E-6, atol=1E-3, rtol=1E-3))

with XDMFFile(MPI.COMM_WORLD, "w.xdmf", "w") as f:
    f.write_mesh(mesh)
    f.write_function(w)
    
with XDMFFile(MPI.COMM_WORLD, "theta.xdmf", "w") as f:
    f.write_mesh(mesh)
    f.write_function(theta)

1.2850646949626487e-06


## Appendix